## Google Drive Setup

In [2]:
folder_id = <Enter ID Here> #workouts folder ID in Google Drive 

In [3]:
credentials_path= <enter json file here>
# Follow the following guide to generate either OAuth or Service Account creds
# https://developers.google.com/workspace/guides/create-credentials

In [4]:
token_file = '~/.credentials/token.json'

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

In [5]:
import os
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError

In [37]:
def authorize_and_print():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print('Requesting new token')
            creds.refresh(Request())
        else:
            print('First run through so putting up login dialog')
            # credentials.json downloaded from https://console.cloud.google.com/apis/credentials
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_file, 'w') as token:
            print('Saving new token')
            print('')
            token.write(creds.to_json())
    else:
        print('Token valid')


    service = build('drive', 'v3', credentials=creds)
    
    # Initialize the Google Docs API service
    docs_service = build('docs', 'v1', credentials=creds)

    # 0. About the user
    results = service.about().get(fields="*").execute()
    emailAddress = results['user']['emailAddress']

    # 1. Call the Drive v3 API
    results = service.files().list(
        pageSize=100,
        fields="nextPageToken, files(id, name)",
        q="'{}' in parents".format(folder_id)  # Query to filter files by parent folder
    ).execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
        return
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))
    return service, docs_service


In [38]:
service, docs_service = authorize_and_print()

Token valid
Files:
Intentions (1EhL5Ao7IXgQckDN1Ig3wt5Uq7LczAxe6-8aTzE1MRBM)
2024-02-12 (1lqQEFqsmpcftbKDciSICv4qN4HaM9T8dIjFr9Nh7F6s)
Reference Exercises (1dKFus6qAbF1FX17FKik8AsLJWck5Tru96fDO_mE4b3U)
2024-02-08 (1QLQ63lqM2A6-Bim0hl1w_ia4DdyD8juNkYvDTvaz3Xk)
2023-08-09 (1dVZxmmtxbKZtMSoiXyV8kXb5smjramS9HKTq9pDQazI)


## Google Doc Creation Tool

In [9]:
def create_google_doc(service, file_name='test_creation', folder_id=folder_id):
    file_metadata = {
        'name': file_name,
        'mimeType': 'application/vnd.google-apps.document'
    }
    if folder_id:
        file_metadata['parents'] = [folder_id]

    file = service.files().create(body=file_metadata, fields='id').execute()
    print(f"Created Google Doc with ID: {file.get('id')}")
    return file.get('id')


In [10]:
def insert_text(document_id, text, index=1):
    requests = [
        {
            'insertText': {
                'location': {
                    'index': index,
                },
                'text': text
            }
        }
    ]

    result = docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    print(f"Text inserted into document with ID: {document_id}")

In [11]:
# Import the tool decorator
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def create_new_doc_with_text(input_data):
    """Create a new Google Doc with specified text as input, file_name as a parameter
    "input_text": "Here is some text for the document: a structured workout log",
    "file_name": "YYYY-MM-DD",
    ."""
    input_text = input_data['input_text']
    file_name = input_data['file_name']
    doc_id = create_google_doc(service, file_name)
    return insert_text(doc_id, input_text)

## Retrieval Tool

In [12]:
# implementing https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents
from dotenv import find_dotenv, load_dotenv
from langchain.document_loaders import GoogleDriveLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

# Load environment variables from .env file
load_dotenv(find_dotenv())

True

In [13]:
# Initialize Google Drive loader
loader = GoogleDriveLoader(
    folder_id=folder_id, #workouts nickmarveaux@gmail.com. 
    credentials_path=credentials_path,
    recursive=False
)
docs = loader.load()


In [14]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [15]:
retriever = db.as_retriever()

## Formatted Date Tool

In [16]:
from datetime import datetime
import pytz

@tool
def get_todays_date_nyc():
    """Get today's date in NYC (the user's hometown) in YYYY-MM-DD format."""

    # Define the Eastern Time Zone
    eastern = pytz.timezone('America/New_York')
    
    # Get the current datetime in Eastern Time
    datetime_nyc = datetime.now(eastern)
    
    # Format the date as YYYY-MM-DD
    return datetime_nyc.strftime('%Y-%m-%d')

## Combining all the tools

In [20]:
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import BaseTool, StructuredTool, Tool, tool, DuckDuckGoSearchRun


name = "search_past_workouts"
desc = "searches past workouts and the overall intenations (file: intentions)"
document_retrieval_tool = create_retriever_tool(
    retriever,
    name,
    desc
)
search = DuckDuckGoSearchRun()

internet_search_tool = Tool.from_function(
    func=search.run,
    name="internet_search",
    description="useful for when you need to search the internet for information"
)
tools = [document_retrieval_tool, internet_search_tool, get_todays_date_nyc, create_new_doc_with_text]

## Prompt Engineering

In [17]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages[0].prompt

PromptTemplate(input_variables=[], template='You are a helpful assistant')

In [18]:
prompt.messages[0].prompt.template = """
You are a helpful workout assistant. Use your get_todays_date_nyc to find today's date.

You will output workout logs, using a tool to save them to new file whose name is the date YYYY-MM-DD.

You will only log exercises whose name exactly matches the name of exercises in the Reference Exercises doc.

You can try to coach the user, using their personal preferences as saved in the Preferences document.

If a user asks if there are any workouts stored for a previous date, check the title of the document: 
the title has to exactly match the date in format YYYY-MM-DD for that to count as a valid log of that day's workout.
Logging past exercises is also a feature you support"""


In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.4, model='gpt-4-0125-preview')
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
result = agent_executor.invoke({"input": "hi, im bob"})



> Entering new AgentExecutor chain...
Hello, Bob! How can I assist you with your workout today?

> Finished chain.


In [23]:
msg = "Are there any workouts stored for today? What is today's date?"


In [28]:
result['output']

'I found workouts stored for the following dates:\n\n- February 8, 2024\n- February 12, 2024\n- August 9, 2023\n\nThese logs contain detailed workout routines, including exercises, repetitions, sets, and additional notes on locations and specific exercise circuits.'

In [26]:
msg = "What dates do you have workouts stored for?"


In [171]:
msg = "Are there any workouts stored for August 9 2023?"

In [ ]:
msg = """
Log the following workout for Thursday Feb 8th 2024, which is not today (in the past). Note: day-off from XC ski, Sjusjoen Cabin
Workout: all Christiania Exercises circuit
PT: 20 tibialis raises
20x Calf raises per side
90second balance
Half circle foot pointed, point raises
Heria 2023 abs all exercises 
100 pushups (4x 25)"""

In [29]:
msg = "I want to log an exercise called Bench Press. Is there any reference exercise similar to that?"

In [35]:
result['output']

'Based on the established schedule, starting with January 2024 as Schedule A and alternating each month, April 2024 would be designated as Schedule B. This means for your workouts in April 2024, you should follow the guidelines for Schedule B, which include:\n\n- Performing approximately 8-15 repetitions per set with moderate to lighter weights.\n- Completing 2-3 sets per exercise.\n- Taking about 90 seconds of rest between sets.\n\nThis schedule is part of a periodization plan to optimize strength and muscle hypertrophy by alternating the volume and intensity of your workouts. Schedule B focuses more on endurance and muscle definition by using lighter weights and higher repetitions compared to Schedule A, which emphasizes heavier weights and lower repetitions for building strength.'

In [33]:
msg = "Based on my preferences / intentions in terms of schedule A and B (low/high volume), What Schedule of workout am I doing April 2024?"

## Future Directions

* Improve UI (simple to add this to Slack, tutorials exist). 
* Function for inputting logs automatically re-initializes the retrieval tool. 
* "Code Intepreter" Environment for creating data visualizations.  